In [1]:
import pandas as pd
import unicodedata as ud
import re
from sklearn.model_selection import train_test_split
import time
import string
import sys, traceback, logging
import matplotlib.pyplot as plt
import json
from bs4 import BeautifulSoup
from markdown import markdown
import warnings
warnings.filterwarnings('ignore')
from collections import Counter
from nltk.corpus import wordnet as wn
from nltk.corpus import words as w
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
import seaborn as sns
from sklearn import preprocessing
# Init Conf
pd.set_option('display.max_colwidth', 1000)

In [2]:
# def stem(text):
#     tokens = word_tokenize(text)
#     result = ' '.join([str(ps.stem(token)) for token in tokens if token not in stops])  
# #     result = ' '.join([token for token in tokens if token not in stops])
#     sentences = sent_tokenize(result)
#     return (result, sentences)
    
def extract_code(body):
    #closing open code sections
    if(len(triple_backtick.findall(body))//2 == 1):
        body += "```"
        
    code = ""
    for code_regex in code_regexes:
        code_sections = code_regex.findall(body)
        code += " "
        code += " ".join(code_sections)
        for code_section in code_sections:
            body = body.replace(code_section, ' codereplaced ')
    return(code, body)

def extract_url(body):
    urls = url_regex.findall(body)
    body_urls = ""
    body_urls += " ".join(urls)
    for url in urls:
        body = body.replace(url, ' urlreplaced ')
    return(body_urls, body)

def extract_email(body):
    emails = email_regex.findall(body)
    body_emails = ""
    body_emails += " ".join(emails)
    for email in emails:
        body = body.replace(email, ' emailreplaced ')
    return(body_emails, body)

def camel_case_split(str): 
    words = [[str[0]]] 
    for c in str[1:]: 
        if words[-1][-1].islower() and c.isupper(): 
            words.append(list(c)) 
        else: 
            words[-1].append(c) 
    return ' '.join([''.join(word) for word in words])

def deal_with_camel_cases(text):
    #it was tokenize, changed to split in order to keep @codes together
    #previous one changed it to @ code, I'am to I 'am
    tokens = text.split()
    result = ' '.join([camel_case_split(token) for token in tokens])  
    return result

def relace_words_list(text, wordlist):    
    _ = []
    for word in text.split():
        if word.lower() in wordlist:
            _.append(wordlist[word.lower()])
        else:
            _.append(word.lower())
    return " ".join(_)

In [3]:
punctuations = '!"$%&\()*,/:;<=>[\\]^`{|}~+'#omitted ' for contractions, excluded # @ . ? -
replace_string = ' '*len(punctuations)
num_of_alpha_char_regex = re.compile(r'[a-zA-Z_]')
#url_regex = r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))'''
url_regex = re.compile(r'\(?https?://[^\s<>"]+|www\.[^\s<>"]+')
email_regex = re.compile( r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+')
triple_backtick = re.compile(r'```')
code_regexes = [re.compile(r'```.*?```',re.DOTALL), re.compile(r'``.*?``',re.DOTALL), re.compile(r'`.*?`',re.DOTALL)]
word_regex = re.compile(r'\w+')
ascii_regex = re.compile(r'[^\x00-\x7f]')
date_regex = re.compile(r'\d{4}-\d{2}-\d{2}')
time_regex = re.compile(r'\d{2}:\d{2}:\d{2}')
issue_regex = re.compile(r'#[0-9]+')
#start with a / or start with a word and continue having /. also covers ../, ././, ...
path_regex = re.compile(r'([a-zA-Z0-9._-]+)?(?:\.{2})?(?:\/\.{2})*((\/){1,2}[a-zA-Z0-9._-]+)+')
function_regex = re.compile(r'[a-zA-Z][a-zA-Z0-9_.]*\([a-zA-Z0-9_, ]*\)')
literal_nonascii_char_regex = re.compile(r'\\x[0-9]{2,}')#literally \x00 \xff...
#sinleQoute and dash at the start of sententence
qoute_starter_regex = re.compile(r"^'")
dash_starter_regex = re.compile(r'^-')
dash_qoute_starter_space_regex = re.compile(r"(^| )(-|')+")
space_dash_qoute_end_regex = re.compile(r"(-|')+($| )")
consecutive_dashs_qoute_regex = re.compile(r"[\-|']{2,}")
num_regex = re.compile(r'[0-9]+(th)?(nd)?(rd)? ')
#numbers = '0123456789'
numbers = re.compile(r'\b([A-Za-z]*)\d+\b|\b\d+([A-Za-z]*)\b')

In [4]:
contractions = {
    "'ll": "will",
    "n't": "not",
    "'ve": "have",
    "'m": "am",
    "'s": "is",
    "doesnt": "does not",
    "dont": "does not",
    "cant": "does not",    
    "wont": "will not",
    "let's": "let us",#####-----------------------
    "ain't": "are not",#am not / 
    "aren't": "are not",# / am not
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had",# / he would
    "he'd've": "he would have",
    "he'll": "he will",#he shall / 
    "he'll've": "he will have",#he shall have / 
    "he's": "he is",#he has / 
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",#how has / 
    "i'd": "I would",#I had / 
    "i'd've": "I would have",
    "i'll": "I will",#I shall / 
    "i'll've": "I will have",#I shall have / 
    "i'm": "I am",
    "i've": "I have",
    "isn't": "is not",
    "it'd": "it had",# / it would
    "it'd've": "it would have",
    "it'll": "it will",#it shall / 
    "it'll've": "it will have",
    "it's": "it is",# / it has, it shall have / 
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she had",# / she would
    "she'd've": "she would have",
    "she'll": "she will",#she shall / 
    "she'll've": "she will have",#she shall have / 
    "she's": "she is",#she has / 
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",#so as / 
    "that'd": "that had",#that would / 
    "that'd've": "that would have",
    "that's": "that is",#that has / 
    "there'd": "there would",#there had / 
    "there'd've": "there would have",
    "there's": "there is",#there has / 
    "they'd": "they would",#they had / 
    "they'd've": "they would have",
    "they'll": "they will",#they shall / 
    "they'll've": "they will have",#they shall have / 
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",#we had / 
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",#what shall / 
    "what'll've": "what will have",#what shall have / 
    "what're": "what are",
    "what's": "what is",#what has / 
    "what've": "what have",
    "when's": "when is",#when has / 
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",#where has / 
    "where've": "where have",
    "who'll": "who will",#who shall / 
    "who'll've": "who will have",#who shall have / 
    "who's": "who is",#who has / 
    "who've": "who have",
    "why's": "why is",#why has / 
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",#you had / 
    "you'd've": "you would have",
    "you'll": "you will",#you shall / 
    "you'll've": "you will have",#you shall have / 
    "you're": "you are",
    "you've": "you have"
}

In [5]:
abrv_top50 = {   "2moro":"tomorrow","2nite":"tonight", 
                 "btw":"moreover",#"by the way",#moreover                   
                 "gr8":"great", 
                 "imho":"opinion",                
                 "j/k":"kidding", 
                 "lmao":"laughing",    
                 "lol":"laughing",                 
                 "nub":"new person", 
                 "noob":"new person", 
                 "newbe":"new person", 
                 "omg":"disbelief", #"Oh my god", 
                 "pov":"point",#"point of view", 
                 "plz":"please",
                 "thx":"thanks",  
                 "thks":"thanks", 
                 "rtm":"read manual", 
                 "rtfm":"read manual", 
                 "wtf":"annoyment",#"what the fuck" 
                 "i.e.": "meaning",
                 ".i.e.": "meaning",
                 "ie": "meaning",
                 "e.g.": "example",
                 ".e.g.": "example",
                 "eg": "example"
                }
                 #"rt":"RealTime", "irl":"In Real Life", "fwiw":"for what it is worth","mhoty":"My Hat is Off To You", 
                 #"TLC":"Tender Loving Care",#"BRB":"Be Right Back",
                 #"tmi":"Too MuchI nformation",#"B4N":"Bye For Now",                  
                 #"BCNU":"Be Seeing You", 
                 #"BFF":"Best Friends Forever", 
                 #"CYA":"Cover Your Ass", 
                 #"DBEYR":"Don't Believe Everything You Read",      
                 #"DILLIGAS":"Do I Look Like I Give A Shit", 
                 #"FUD":"Fear, Uncertainty and Disinformation", 
                 #"TTYL":"Talk To You Later", 
                 #"TYVM":"Thank You Very Much",#"ILY":"I Love You", 
                 #"VBG":"Very Big Grin",#"ISO":"In Search Of", 
                 #"WEG":"Wicked Evil Grin",
                 #"WYWH":"Wish You Were Here",#"LYLAS":"Love You Like A Sister", 
                 #"XOXO":"Hugs and Kisses"
                 #"SITD":"Still In The Dark",#"NIMBY":"Not In My BackYard", 
                 #"NP":"No Problem", 
                 #"SOL": "Sooner Or Later", 
                 #"STBY":"Sucks To Be You",
                 #"SWAK":"Sealed With A Kiss",#"OIC":"Oh, I See",
                 #"TFH":"Thread From Hell",#"OT":"Off Topic",
                 #"TX":"Thanks",#"RBTL":"Read Between The Lines", 
                 #"ROTFLMAO":"Rolling On The Floor Laughing My Ass Off", 

In [6]:
#added plural form as well
abrv_CS_common = { 
    "alloc": "allocate",
    "auto": "automatic",
    "arg": "argument",
    "args": "argument",
    "attr": "attribute",
    "app": "application",
    "apps": "application",
    "async": "asynchronous",
    "bg": "background",
    "bin": "binary",
    "bool": "boolean",
    "btn": "button",
    "calc": "calculate",
    'clr': "clear",
    'cmd': "command",
    "cmp": "compare",
    'cnt': "counter",
    "cont": "continue",
    "config": "configuration",
    "cfg": "configuration",
    'col': "column",
    "ctrl": "control",
    "cpy": "copy",
    "db": 'database',
    "docs": 'documents',
    "doc": 'documents',
    "dev": "device",
    "disp": "display",
    "env": "environment",
    "err": "error",
    "func": "function",
    "foo": "function",
    "img": "image",
    "info": "information",
    #"infos": "information",
    "init": "initialize",
    "lib": "library",
    "gen": "generate",
    "math": "mathematics",
    "mcu": "microcontroller",
    "mid": "middle",
    "misc": "miscellaneous",
    "mng": "manager",
    "msg": "message",
    "notif": "notification",
    "num": "number",
    "obj": "object",
    "oo": "object-oriented",
    "param": "parameter",
    "params": "parameter",
    "pic": "picture",
    "prev": "previous",
    "pkg": "package",
    "ptr": "pointer",
    "px": "pixel",
    "src": "source",
    "spec": "specification",
    "seq": "sequence",
    "sync": "synchronous",
    "temp": "temporary",
    "tgl": "toggle",
    "tmr": "timer",
    "txt": "text",
    "v": "version",
    "ver": "version",
    "vb": "visual basic",
    "vm": "virtual machine",
    "val": "value",
    "var": "variable",
    "vert": "vertical",
    "win": "windows"}

In [7]:
#replace months and date which are in the regex format of date
dates_list = {'january': 'datereplaced','february': 'datereplaced','march': 'datereplaced',
              'april': 'datereplaced','june': 'datereplaced','july': 'datereplaced','august': 'datereplaced',
              'september': 'datereplaced','october': 'datereplaced','november': 'datereplaced','december': 'datereplaced',
                        'tuesday':'dayreplaced', 'wednesday':'dayreplaced', 'thursday':'dayreplaced',
              'friday':'dayreplaced', 'saturday':'day', 'sunday':'dayreplaced', 'monday':'dayreplaced',
              'afternoon':'timereplaced', 'tonight':'timereplaced', 'night':'timereplaced', 'morning': 'timereplaced',
              'tomorrow': 'timereplaced',"o'clock": 'timereplaced'}

digits_list = {'one': 'digitreplaced','two': 'digitreplaced','three': 'digitreplaced','four': 'digitreplaced',
              'five': 'digitreplaced','six': 'digitreplaced','seven': 'digitreplaced','eight': 'digitreplaced',
              'nine': 'digitreplaced','ten': 'digitreplaced','eleven': 'digitreplaced','twenty': 'digitreplaced',
              'hundred': 'digitreplaced','thousand': 'digitreplaced','million': 'digitreplaced','billion': 'digitreplaced'}

In [8]:
#omitting not from stop words #hack #change later
nltk_stopwords = ['i',
 'me','my','myself','we','our','ours','ourselves','you',"you're","you've",
 "you'll","you'd",'your','yours','yourself','yourselves',
 'he','him','his','himself',
 'she',"she's",'her','hers','herself',
 'it',"it's",'its','itself',
 'they','them','their','theirs','themselves',
 'what','which','who','whom',
 'this','that',"that'll",'these','those',
 'am','is','are','was','were',
 'be','been','being',
 'have','has','had','having',
 'do','does','did','doing',
 'a','an','the','and','but','if','or','of',
 'because','as','until','while',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 #'not',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 "don't",
 'should',
 "should've",
 'now',
 'd',
 'll',
 'm',
 'o',
 're',
 've',
 'y',
 'ain',
 'aren',
 "aren't",
 'couldn',
 "couldn't",
 'didn',
 "didn't",
 'doesn',
 "doesn't",
 'hadn',
 "hadn't",
 'hasn',
 "hasn't",
 'haven',
 "haven't",
 'isn',
 "isn't",
 'ma',
 'mightn',
 "mightn't",
# 'mustn',
# "mustn't",
 'needn',
 "needn't",
 'shan',
 "shan't",
 'shouldn',
 "shouldn't",
 'wasn',
 "wasn't",
 'weren',
 "weren't",
 'won',
 "won't",
 'wouldn',
 "wouldn't"]

In [9]:
      
my_stopwords = ['use', 'get', 'would', 'etc', 'example', 'also', 'could', 'please', 'thank', 'thanks','seem', 
                   'instead', 'hi', 'think', 'something', 'may', 'without', 'still', 'sure', 'however', 
                  'since', 'mean', 'already', 'done', 'well', 'another', 'might', 'via', 
 'maybee', 'us', 'always', 'let', 'hello', 'given', 'per', 'either', 'got', 'due', 'within', 
 'shown', 'sometime','sometimes', 'ok','become', 'otherwise', 'else', 'other', 'therefore', 'anymore', 'although', 
 'often', 'today', 'upon','anyway', 'sorry', 'along', 'foo', 'together', 'across', 'whenever', 
 'somewherer', 'hey', 'somehow', 'unless', 'behind', 'saw', 'almost', 'whatever', 'welcome', 'everyone', 
 'went','earlier','accordingly', 'shall', 'prior', 'ago', 'came','ever', 'cool', 'mostly', 'obvious', 'obviously', 
 'clearly', 'onto', 'anywhere', 'neither', 'hence', '``', '`',  'oh', 'oops','somewhat',
'.', 'x', "'", 'c', 'e', 'b', 'v', 'f', 'r', 'j', '-', 'h', 'z', 'l', 
 'p', 'n', 'k', 'w', 'u', 'q', 'bb', 'xx', 'ff', 'fff', 'god', "'ve", "'m","'s",'ie', 'th', 'nd' 'faq', 'db']


 #'ie', 'th', 'nd' 'faq', 'db''tech'
 #'doesnt', 'cant', 'dont', "n't" 
# '#','@', '?', 


#obligations
#'must', 
extra_stop_words_list = my_stopwords + nltk_stopwords

In [10]:
df = pd.read_csv("data/english_issues.csv")
print(df.shape)

(871878, 32)


In [11]:
df['label_cat'].value_counts()

bug        418522
feature    363321
other       90035
Name: label_cat, dtype: int64

In [12]:
#omit empty records
#dropping issues without title or body

df = df[~df['title'].isna()]
df = df[~df['body'].isna()]
df = df[df['title']!='']
df = df[df['body']!='']
df = df[~df['title'].isnull()]
df = df[~df['body'].isnull()]

#set the types
df.title = df.title.astype(str)
df.body = df.body.astype(str)

print(df.shape)

(747057, 32)


In [13]:
df['title_words_num'] = df['title'].apply(lambda x:len(x.split()))
df['body_words_num'] = df['body'].apply(lambda x:len(x.split()))

In [ ]:
#make a copy to start processing
df['title_processed'] = df['title']
df['title_processed'] = df['title_processed'].astype(str)
df['body_processed'] = df['body']
df['body_processed'] = df['body_processed'].astype(str)

#apply changes to title and body to make it appropriate for detecting language+word distributations
print('Converting camel cases...')
df['title_processed'] = df['title_processed'].apply(lambda x:deal_with_camel_cases(x))
df['body_processed'] = df['body_processed'].apply(lambda x:deal_with_camel_cases(x))

#to lower
print('Converting to lower case...')
df['title_processed'] = df['title_processed'].str.lower()
df['body_processed'] = df['body_processed'].str.lower()

In [ ]:
#remove issues with speical titles
df = df[~df.title_processed.str.contains("not an issue", na=False)]
df = df[~df.title_processed.str.contains("not a issue", na=False)]
df = df[~df.title_processed.str.contains("its not me", na=False)]
df = df[~df.title_processed.str.contains("it's not me", na=False)]
df = df[~df.title_processed.str.startswith("oh my god", na=False)]
df = df[~df.title_processed.str.startswith("omg", na=False)]
# issues = issues[~issues.title.str.startswith("test", na=False)]
print(df.shape)

In [ ]:
# Remove words less than 3 char
#it should not be here, bcuz it omits ** or ``` for code...
# print('Removing words with less than 3 characters...')
# df['raw_title'] = df['raw_title'].str.findall('\b\w{1,3}\b').str.join(' ')
# df['raw_body'] = df['raw_body'].str.findall('\b\w{1,3}\b').str.join(' ')

#drop issues with title or body length less than 3        
df = df[~(df['title_processed'].str.len()<3)]
df = df[~(df['body_processed'].str.len()<3)]

print(df.shape)

In [ ]:
#replace @code/@date/@time/@url/@path/@email before omitting numbers/special characters/puntuations

#omit code snippets from title, if any.
#omit codes before markdown, since beautifulsoup omits '''
# print('Replacing code snippets...')
# for regex in code_regexes:
#     df['title_processed'] = df['title_processed'].apply(lambda x:regex.sub(" codereplaced ",x))

temp = df['title_processed'].apply(lambda x:extract_code(x))
code, title = zip(*temp)
df['title_processed'] = list(title)

#extract code from body
temp = df['body_processed'].apply(lambda x:extract_code(x))
code, body = zip(*temp)
df['code'] = list(code)
df['body_processed'] = list(body)

#replacing functions
print('Replacing functions...')
df['title_processed'] = df['title_processed'].apply(lambda x:function_regex.sub(" functionreplaced ",x))
df['body_processed'] = df['body_processed'].apply(lambda x:function_regex.sub(" functionreplaced ",x))

#replacing urls and emails; extracting them from body
print('Replacing emails and urls...')
df['title_processed'] = df['title_processed'].apply(lambda x:url_regex.sub(" urlreplaced ",x))
df['title_processed'] = df['title_processed'].apply(lambda x:email_regex.sub(" emailreplaced ",x))
#from body
temp = df['body_processed'].apply(lambda x:extract_url(x))
url, body = zip(*temp)
df['url'] = list(url)
df['body_processed'] = list(body)
temp = df['body_processed'].apply(lambda x:extract_email(x))
email, body = zip(*temp)
df['email'] = list(email)
df['body_processed'] = list(body)

#replace date and time
print('Replacing dates and times...')
df['title_processed'] = df['title_processed'].apply(lambda x:date_regex.sub(" datereplaced ",x))
df['title_processed'] = df['title_processed'].apply(lambda x:time_regex.sub(" timereplaced ",x))
df['body_processed'] = df['body_processed'].apply(lambda x:date_regex.sub(" datereplaced ",x))
df['body_processed'] = df['body_processed'].apply(lambda x:time_regex.sub(" timereplaced ",x))

#replace paths/directories
print('Replacing paths...')
df['title_processed'] = df['title_processed'].apply(lambda x:path_regex.sub(" pathreplaced ",x))
df['body_processed'] = df['body_processed'].apply(lambda x:path_regex.sub(" pathreplaced ",x))

#replace isse numbers as #anynumber
print('Replacing issue numbers...')
df['title_processed'] = df['title_processed'].apply(lambda x:issue_regex.sub(" issuereplaced ",x))
df['body_processed'] = df['body_processed'].apply(lambda x:issue_regex.sub(" issuereplaced ",x))

#dropping issues without title or body
print('Clean up...')
df = df[~df['title_processed'].isna()]
df = df[~df['body_processed'].isna()]
df = df[df['title_processed']!='']
df = df[df['body_processed']!='']
df = df[~df['title_processed'].isnull()]
df = df[~df['body_processed'].isnull()]
print(df.shape)

In [ ]:
# #Remove Markdown Language such as **, ##, ...
# #Join them with space, otherwise words would attach
# #this is not complete, the next one is better
# def mark_to_plain(t):
#     html = markdown(t)
#     return ' '.join(BeautifulSoup(html).findAll(text=True))

# print('Removing markdowns in titles...')
# df['title_processed'] = df['title_processed'].apply(mark_to_plain)
# print('Removing markdowns in bodies...')
# df['body_processed'] = df['body_processed'].apply(mark_to_plain)


#Remove Markdown Language such as **, ##, ...
#Join them with space, otherwise words would attach
#this is not complete, the next one is better
#I manually added title consecutive # and *

def markdown_to_text(markdown_string):
    # md -> html -> text since BeautifulSoup can extract text cleanly
    html = markdown(markdown_string)

    # remove code snippets
    html = re.sub(r'<pre>(.*?)</pre>', ' ', html)
    html = re.sub(r'<code>(.*?)</code >', ' ', html)

    html = re.sub(r'[#]{2,6}', ' ', html)
    #html = re.sub(r'[*]+', ' ', html)

    # extract text
    soup = BeautifulSoup(html, "html.parser")
    text = ''.join(soup.findAll(text=True))

    return text

print('Removing markdowns in desc_processed...')
df['title_processed'] = df['title_processed'].apply(markdown_to_text)
print('Removing markdowns in readme_processed...')
df['body_processed'] = df['body_processed'].apply(markdown_to_text)

In [ ]:
replace_string = ' '*len(punctuations)
#removing punctuations before replacing in the next sections, bcuz sth like (e.g. nextWord) won't be replaced otherwise
print('Removing punctuations...')
df['title_processed'] = df['title_processed'].str.translate(str.maketrans(punctuations, replace_string))
df['body_processed'] = df['body_processed'].str.translate(str.maketrans(punctuations, replace_string))

#replacing withespaces with space again
print('Replacing white spaces...')
df['title_processed'] = df['title_processed'].apply(lambda x:" ".join(x.split()))
df['body_processed'] = df['body_processed'].apply(lambda x:" ".join(x.split()))

In [ ]:
#dropping issues which number of alphabets appeared in their title is less than 3 or 50% of title length
print('Dropping issues for which number of alphabets (title) is less than 3 or 50% of title length')
df['title_alpha_len'] = df['title_processed'].apply(lambda x:len(num_of_alpha_char_regex.findall(x)))
df = df[df['title_alpha_len'] >= 3]    
df['title_alphabet_ratio'] = df[['title_alpha_len','title_processed']].apply(lambda x:x[0]/len(x[1]), axis=1)

#dropping issues which number of alphabets appeared in their body is less than 3 or 50% of body length
print('Dropping issues for which number of alphabets (body) is less than 3 or 50% of body length')
df['body_alpha_len'] = df['body_processed'].apply(lambda x:len(num_of_alpha_char_regex.findall(x)))
df = df[df['body_alpha_len'] >= 3]    
df['body_alphabet_ratio'] = df[['body_alpha_len','body_processed']].apply(lambda x:x[0]/len(x[1]), axis=1)

#clean up
df = df[df['title_alphabet_ratio'] >= 0.5 ]
df = df[df['body_alphabet_ratio'] >= 0.5 ]

print(df.shape)

In [ ]:
#remove numbers
#do this before replacing contractions, bcuz some of them are stuck with numers and therefore won't be replaced
#bcuz contractions work with words not the whole strings
#that is "e.g.3 something" won't be rplaced
print('Removing numbers...')
#this can be done after removing numbers, in the second step only reomve nd, th, ... 
#but then if sth was a "th" to symbole sth else it will be omitted

#2nd, 3rd, ...
df['title_processed'] = df['title_processed'].apply(lambda x:re.sub(num_regex, ' digitreplaced ', x))
df['body_processed'] = df['body_processed'].apply(lambda x:re.sub(num_regex, ' digitreplaced ', x))

#just replace single numbers (without any letter) or digits right after some letters
df['title_processed'] = df['title_processed'].apply(lambda x:re.sub(numbers, r'\1 digitreplaced \2', x))
df['body_processed'] = df['body_processed'].apply(lambda x:re.sub(numbers, r'\1 digitreplaced \2', x))

# replace_string = ' '*10
# df['desc_processed'] = df['desc_processed'].str.translate(str.maketrans(numbers, replace_string))
# df['readme_processed'] = df['readme_processed'].apply(lambda x:" ".join(x.split()))
                                                    
# df['desc_processed'] = df['desc_processed'].str.translate(str.maketrans(numbers, replace_string))
# df['readme_processed'] = df['readme_processed'].apply(lambda x:" ".join(x.split()))

In [ ]:
#omitting non-ascii charachters
print('Removing non-ascii charachters...')
df['title_processed'] = df['title_processed'].apply(lambda x:re.sub(ascii_regex, '', x))
df['body_processed'] = df['body_processed'].apply(lambda x:re.sub(ascii_regex, '', x))    
df['title_processed'] = df['title_processed'].apply(lambda x:ud.normalize('NFD', x))
df['body_processed'] = df['body_processed'].apply(lambda x:ud.normalize('NFD', x))

#replacing withespaces with space again
print('Replacing white spaces...')
df['title_processed'] = df['title_processed'].apply(lambda x:" ".join(x.split()))
df['body_processed'] = df['body_processed'].apply(lambda x:" ".join(x.split()))

In [ ]:
#periodic cleanup
# Remove White space
print('Removing white spaces...')
df['title_processed'] = df['title_processed'].apply(lambda x:" ".join(x.split()))
df['body_processed'] = df['body_processed'].apply(lambda x:" ".join(x.split()))

#dropping issues without readme or fullname
df = df[~df['title_processed'].isna()]
df = df[df['title_processed']!='']
df = df[~df['title_processed'].isnull()]

df = df[~df['body_processed'].isna()]
df = df[df['body_processed']!='']
df = df[~df['body_processed'].isnull()]

print(df.shape)

In [ ]:
#Replacing '?' to separate them from words. replacing dots in words such as "e.g." make a problem 
#but question marks are ok.
print('Replacing question marks...')
df['title_processed'] = df['title_processed'].str.replace('?', ' ? ') 
df['body_processed'] = df['body_processed'].str.replace('?', ' ? ')

print('Replacing multiple consecutive dots...')
df['title_processed'] = df['title_processed'].str.replace('[\.]{2,}', ' etc ') 
df['body_processed'] = df['body_processed'].str.replace('[\.]{2,}', ' etc ')
                                                        
#omit "_"
print('Removing _ sign...')
df['title_processed'] = df['title_processed'].str.replace('_', ' ')
df['body_processed'] = df['body_processed'].str.replace('_', ' ')

print('Removing dash from start or end of a word or between words. Keep - in the middle of words')
# df['title_processed'] = df['title_processed'].str.replace(" ' ", ' ')
# df['body_processed'] = df['body_processed'].str.replace(" ' ", ' ')

# df['title_processed'] = df['title_processed'].str.replace(' - ', ' ')
# df['body_processed'] = df['body_processed'].str.replace(' - ', ' ')

df['title_processed'] = df['title_processed'].apply(lambda x:re.sub(dash_qoute_starter_space_regex, ' ', x))
df['body_processed'] = df['body_processed'].apply(lambda x:re.sub(dash_qoute_starter_space_regex, ' ', x))

df['title_processed'] = df['title_processed'].apply(lambda x:re.sub(space_dash_qoute_end_regex, ' ', x))
df['body_processed'] = df['body_processed'].apply(lambda x:re.sub(space_dash_qoute_end_regex, ' ', x))

print('Removing multiple consecutive dashs or qoutes....')
df['title_processed'] = df['title_processed'].apply(lambda x:re.sub(consecutive_dashs_qoute_regex, ' ', x))
df['body_processed'] = df['body_processed'].apply(lambda x:re.sub(consecutive_dashs_qoute_regex, ' ', x)) 

print('Removing literal non-ascii characters such as \x00....')
df['title_processed'] = df['title_processed'].apply(lambda x:re.sub(literal_nonascii_char_regex, '', x))
df['body_processed'] = df['body_processed'].apply(lambda x:re.sub(literal_nonascii_char_regex, '', x))  

print('Removing white spaces...')
df['title_processed'] = df['title_processed'].apply(lambda x:" ".join(x.split()))
df['body_processed'] = df['body_processed'].apply(lambda x:" ".join(x.split()))

In [ ]:
#Convert Contractions
print('Replacing contractions...')
df['title_processed'] = df['title_processed'].apply(lambda x:relace_words_list(x, contractions))
df['body_processed'] = df['body_processed'].apply(lambda x:relace_words_list(x, contractions))

In [ ]:
#replace abbreviations
print('Replacing abbreviations...')
df['title_processed'] = df['title_processed'].apply(lambda x:relace_words_list(x, abrv_top50))
df['body_processed'] = df['body_processed'].apply(lambda x:relace_words_list(x, abrv_top50))

In [ ]:
#Replace common abbreviations in computer science and IT
#Do this before markdown, because that will put an space in e.g., I can't and other simialr things

print('Replacing IT abbreviations...')
df['title_processed'] = df['title_processed'].apply(lambda x:relace_words_list(x, abrv_CS_common))
df['body_processed'] = df['body_processed'].apply(lambda x:relace_words_list(x, abrv_CS_common))

In [ ]:
#Replace month day time of day bcuz of different formats of date...
print('Replacing dates...')
df['title_processed'] = df['title_processed'].apply(lambda x:relace_words_list(x, dates_list))
df['body_processed'] = df['body_processed'].apply(lambda x:relace_words_list(x, dates_list))

In [ ]:
#Replace digits
print('Replacing digits...')
df['title_processed'] = df['title_processed'].apply(lambda x:relace_words_list(x, digits_list))
df['body_processed'] = df['body_processed'].apply(lambda x:relace_words_list(x, digits_list))

In [ ]:
#keeping the first sentences before anything
print('Keeping the first sentences before removing words with less than 3 chars...')
df['title_processed_sentences'] = df['title_processed'].str.split('.') 
df['body_processed_sentences'] = df['body_processed'].str.split('.')

In [ ]:
#Replacing dots and question marks to separate them from words
print('Replacing dots...')
df['title_processed'] = df['title_processed'].str.replace('.', ' . ') 
df['body_processed'] = df['body_processed'].str.replace('.', ' . ')

In [ ]:
#add some columns
#computing body and title len
print('Computing body and title len...')
df['body_processed_len'] = df['body_processed'].str.len()
df['title_processed_len'] = df['title_processed'].str.len()
df['title_processed_words_num'] = df['title_processed'].apply(lambda x:len(x.split()))
df['body_processed_words_num'] = df['body_processed'].apply(lambda x:len(x.split()))

print('Computing number of different abstract elements...')
df['num_of_sharps'] = df['body_processed'].str.count('#')
df['num_of_at'] = df['body_processed'].str.count('@')
df['num_of_qmark'] = df['body_processed'].str.count('\?')
df['num_of_codesnippets'] = df['title_processed'].str.count('codereplaced')+df['body_processed'].str.count('codereplaced')
df['num_of_functions'] = df['title_processed'].str.count('functionreplaced')+df['body_processed'].str.count('functionreplaced')
df['num_of_issues'] = df['title_processed'].str.count('issuereplaced')+df['body_processed'].str.count('issuereplaced')
df['num_of_paths'] = df['title_processed'].str.count('pathreplaced')+df['body_processed'].str.count('pathreplaced')
df['num_of_dates'] = df['title_processed'].str.count('datereplaced')+df['body_processed'].str.count('datereplaced')
df['num_of_times'] = df['title_processed'].str.count('timereplaced')+df['body_processed'].str.count('timereplaced')
df['num_of_urls'] = df['title_processed'].str.count('urlreplaced')+df['body_processed'].str.count('urlreplaced')
df['num_of_emails'] = df['title_processed'].str.count('emailreplaced')+df['body_processed'].str.count('emailreplaced')
df['num_of_obligations'] = df['title_processed'].str.count('must')+df['body_processed'].str.count('must')

df['has_email'] = df['email'].apply(lambda x: 1 if '@' in x else 0)
df['has_code'] = df['num_of_codesnippets'].apply(lambda x: 0 if x==0 else 1)

In [ ]:
#periodic cleanup
# Remove White space
print('Removing white spaces...')
df['title_processed'] = df['title_processed'].apply(lambda x:" ".join(x.split()))
df['body_processed'] = df['body_processed'].apply(lambda x:" ".join(x.split()))

In [ ]:
#drop issues without title or body
print('Removing empty records..')
df = df[~df['title_processed'].isna()]
df = df[~df['body_processed'].isna()]
df = df[~df['title_processed'].isnull()]
df = df[~df['body_processed'].isnull()]
df = df[df['title_processed']!='']
df = df[df['body_processed']!='']

print(df.shape)

In [ ]:
#extra stemming
print('Removing singleQoute from start or end of a word or between words. Keep - in the middle of words')
df['title_processed'] = df['title_processed'].apply(lambda x:re.sub(dash_qoute_starter_space_regex, ' ', x))
df['body_processed'] = df['body_processed'].apply(lambda x:re.sub(dash_qoute_starter_space_regex, ' ', x))

df['title_processed'] = df['title_processed'].apply(lambda x:re.sub(space_dash_qoute_end_regex, ' ', x))
df['body_processed'] = df['body_processed'].apply(lambda x:re.sub(space_dash_qoute_end_regex, ' ', x))

def replace_extra_stopwords(text, words_list):   
    #first split them then replace
    _ = []
    for word in text.split():
        if word.lower() in words_list:
            _.append('')
        else:
            _.append(word.lower())
    return " ".join(_)

print('Replacing extra stop words in title_processed...')
df['title_processed'] = df['title_processed'].apply(lambda x:replace_extra_stopwords(x, extra_stop_words_list))
print('Replacing extra stop words in body_processed...')
df['body_processed'] = df['body_processed'].apply(lambda x:replace_extra_stopwords(x, extra_stop_words_list))

In [ ]:
print('Removing singleQoute from start or end of a word or between words. Keep - in the middle of words')      
#replacing dashs and qoutes from the start
df['title_processed'] = df['title_processed'].apply(lambda x:re.sub(dash_qoute_starter_space_regex, ' ', x))
df['body_processed'] = df['body_processed'].apply(lambda x:re.sub(dash_qoute_starter_space_regex, ' ', x))

df['title_processed'] = df['title_processed'].apply(lambda x:re.sub(space_dash_qoute_end_regex, ' ', x))
df['body_processed'] = df['body_processed'].apply(lambda x:re.sub(space_dash_qoute_end_regex, ' ', x))

#remove # and @ to lower word frequency
df['title_processed'] = df['title_processed'].apply(lambda x:re.sub('#', '', x))
df['body_processed'] = df['body_processed'].apply(lambda x:re.sub('#', '', x))

df['title_processed'] = df['title_processed'].apply(lambda x:re.sub('@', '', x))
df['body_processed'] = df['body_processed'].apply(lambda x:re.sub('@', '', x))

print('Replacing dash...')
df['title_processed'] = df['title_processed'].str.replace('-', ' ') 
df['body_processed'] = df['body_processed'].str.replace('-', ' ')

print(df.shape)

In [ ]:
print('Removing white spaces...')
df['title_processed'] = df['title_processed'].apply(lambda x:" ".join(x.split()))
df['body_processed'] = df['body_processed'].apply(lambda x:" ".join(x.split()))

print('Removing empty records..')
df = df[~df['title_processed'].isna()]
df = df[~df['body_processed'].isna()]
df = df[df['title_processed']!='']
df = df[df['body_processed']!='']
df = df[~df['title_processed'].isnull()]
df = df[~df['body_processed'].isnull()]

print(df.shape)

In [ ]:
#second replace to make sure everything is ok
#just replace single numbers (without any letter) or digits right after/before some letters
df['title_processed'] = df['title_processed'].apply(lambda x:re.sub(numbers, r'\1 digitreplaced \2', x))
df['body_processed'] = df['body_processed'].apply(lambda x:re.sub(numbers, r'\1 digitreplaced \2', x))

In [ ]:
#lemmatiziation
stops_list = extra_stop_words_list

lemmatizer = WordNetLemmatizer() 

def lemmatize(text):
    tokens = word_tokenize(text)
    result = ' '.join([str(lemmatizer.lemmatize(token)) for token in tokens if token not in stops_list])  
    sentences = sent_tokenize(result)
    return result

df['title_proc_lem'] = df['title_processed'].apply(lambda x:lemmatize(str(x)))
df['body_proc_lem'] = df['body_processed'].apply(lambda x:lemmatize(str(x)))

df['title_lem_len'] = df['title_proc_lem'].apply(lambda x: len(x))
df['title_lem_words_num'] = df['title_proc_lem'].apply(lambda x:len(x.split()))
df['body_lem_len'] = df['body_proc_lem'].apply(lambda x: len(x))
df['body_lem_words_num'] = df['body_proc_lem'].apply(lambda x:len(x.split()))

In [ ]:
#finding number of issue label categories
df["lower_labels"] = df.labels.apply(lambda x: str(x).lower())
df["bug_label"] = df.lower_labels.apply(lambda x: 1 if "bug" in x else 0)
df["feature_label"] = df.lower_labels.apply(lambda x: 1 if ("feature" in x or "enhancement" in x) else 0)
df["other_label"] = df.lower_labels.apply(lambda x: 1 if ("support" in x or "docs" in x or "documentation" in x or "question" in x) else 0)
df["mono_label"] = df["bug_label"] + df["feature_label"] + df["other_label"]
df["mono_label"].value_counts()

In [ ]:
#filtering out issues with more than one label cat
df_monolabel = df[df.mono_label == 1]
df = []
df_monolabel.shape

In [ ]:
#filtering out inconsistent labels
df_monolabel = df_monolabel[((df_monolabel.other_label == 1) & (df_monolabel.label_cat == "other")) | ((df_monolabel.bug_label == 1) & (df_monolabel.label_cat == "bug")) | ((df_monolabel.feature_label == 1) & (df_monolabel.label_cat == "feature"))]
df_monolabel.drop(columns=["bug_label", "feature_label", "other_label", "mono_label"], inplace=True)
df_monolabel.shape

In [ ]:
df_monolabel.label_cat.value_counts()

In [ ]:
#splitting data to train and test
y = df_monolabel["label_cat"]
train, test = train_test_split(df_monolabel, test_size=0.2, random_state = 42, stratify=y, shuffle=True)
df_monolabel = []
train["test_tag"] = 0
test["test_tag"] = 1
df = pd.concat([train, test], ignore_index=True)

In [ ]:
#saving clean dataset
textonly_columns = ["number", "repository_url", "title_processed", "body_processed", "label_cat", "test_tag"]

def save_dataset(df, name):
    df.to_csv(f"data/{name}.csv", index=False)
    df[textonly_columns].to_csv(f'data/{name}_textonly.csv', index=False)
    
save_dataset(df, "clean_mono_norm")

In [ ]:
# plt.figure(figsize=(15,8))
# sns.distplot(df_monolabel['title_processed_words_num'], bins =30)

In [ ]:
# plt.figure(figsize=(15,8))
# sns.distplot(df_monolabel[df_monolabel.body_processed_words_num<5000]["body_processed_words_num"])

In [ ]:
# plt.figure(figsize=(15,8))
# sns.distplot(df_monolabel[df_monolabel.title_processed_words_num<60].title_processed_len)

In [ ]:
# plt.figure(figsize=(15,8))
# sns.distplot(df_monolabel[df_monolabel.body_processed_len<10000].body_processed_len)

In [ ]:
# df_monolabel[(df_monolabel.title_processed_words_num<60)][df_monolabel.title_processed_len>350].shape

In [ ]:
# df_monolabel[(df_monolabel.body_processed_words_num<2000)][df_monolabel.body_processed_len>6000].shape

In [ ]:
# #removing outliers
# short_df = df_monolabel[(df_monolabel['title_processed_words_num'] <= 60) &
#                         (df_monolabel['body_processed_words_num'] <= 2000) &
#                         (df_monolabel['title_processed_len'] <= 350) &                                        
#                         (df_monolabel['body_processed_len'] <= 6000)]
# save_dataset(short_df,"short")
# short_df.shape

In [ ]:
# #removing issues that their title or body contains only one word
# multi_word_df = df_monolabel[df_monolabel.title_processed_words_num > 1]
# multi_word_df = df_monolabel[df_monolabel.body_processed_words_num > 1]
# df_monolabel = []
# save_dataset(multi_word_df,"multi")
# multi_word_df.shape

In [ ]:
# #removing outliers AND single word issues
# short_multi_df = short_df[short_df.title_processed_words_num > 1]
# short_multi_df = short_df[short_df.body_processed_words_num > 1]
# multi_word_df = []
# short_df = []
# save_dataset(short_multi_df,"short_multi")
# short_multi_df.shape